In [3]:
import numpy as np
import math
import random
import timeit
from scipy.special import expit

data = np.loadtxt('heart.dat')

for k in range(len(data)):
    if data[k][13]==2:
        data[k][13]=1 
    else:
        data[k][13]=-1
        
data_x = data[:220]
data_t = data[220:]
dtype_ = np.dtype(dtype="float64")

input_data=[]

for r in range(len(data_x)):
    input_data.append([data_x[r][:12],data_x[r][13:]])

testing_data=[]

for s in range(len(data_t)):
    testing_data.append([data_t[s][:12],data_t[s][13:]])

#Setting up the data type for numpy arrays
dtype="float64"
dtype_ = np.dtype(dtype)
x_i = list()
y_i = list()
for i, j in input_data:
        x_i.append(i)
        y_i.append(j)
x_i, y_i = np.array(x_i, dtype=dtype_), np.array(y_i, dtype=dtype_)


#Initializing the values of the Omega and Beta
start_data_w= np.random.randint(np.size(x_i[0]), size=np.size(x_i[0]))
start_data_b = np.random.randint(np.size(y_i))

#Gradient function
def gradient(training,W,B):
    L = 0.0001
    m = np.size(W)
    G0 = 2*L*W
    
    for x,y in input_data:
        #New array with the specified precision
        z = np.sum(-y*(np.sum(W*x[:, None]) + B))
        z = expit(z)
        sumgrad = np.subtract(np.sum(np.sum(-y*x)*np.exp(z)),np.sum(np.sum(y*np.exp(z))))/np.add(1,np.exp(np.sum(-y*np.exp(z))))
    return(np.add(G0,(1/m)*np.sum(sumgrad)))

def gradient_descent(
    gradient, training_d, start_w, start_b, stepsize, n_iter, tolerance,
    dtype="float64"
):
    start = timeit.default_timer()
    
    #Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    #Setting up the data type for numpy arrays
    dtype_ = np.dtype(dtype)

    #Converting x and y to numpy arrays
    if x_i.shape[0] != y_i.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")

    #Initializing the values of the variables
    vector = np.array(start_w, dtype=dtype_)
    beta = np.array(start_b, dtype=dtype_)

    #Setting up and checking the stepsize
    stepsize = np.array(stepsize, dtype=dtype_)
    if np.any(stepsize <= 0):
        raise ValueError("'stepsize' must be greater than zero")

    #Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    #Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")
    
    #Performing the gradient descent loop
    for _ in range(n_iter):
        #Recalculating the difference
        diff = -stepsize* np.array(gradient(training_d, vector, beta), dtype_)

        #Checking if the absolute difference is small enough
        if np.all(np.abs(diff) > tolerance):
            #Updating the values of the variables
            vector, beta = (val + delta for val, delta in zip((vector, beta), diff))
        else:
            print("break",start,stop)
            break
            
        print(vector,beta)    
    count_accuracy = 0
    
    for h,l in testing_data:
        if np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))>=0 and np.array(l)==1:
                count_accuracy += 1
            
        elif np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))<0 and np.array(l)==-1:
                count_accuracy += 1
            
        check_accuracy = np.divide(count_accuracy,len(testing_data))*100
        
    stop = timeit.default_timer()

    return(vector if vector.shape else vector.item(), stop-start,
           print("The local minimum occurs at: Omega={}".format(vector), "Beta={}".format(beta)),
          print("Reported accuracy:{}%".format(check_accuracy)), print('Run time is: ', stop-start) )

print(gradient_descent(gradient, input_data, start_data_w, start_data_b, 0.0001, 500, 1e-04))

[ 8.99921596e+00  2.99921596e+00  6.99921596e+00  1.09992160e+01
 -7.84038938e-04  7.99921596e+00  9.99921596e+00 -7.84038938e-04
  3.99921596e+00  8.99921596e+00 -7.84038938e-04 -7.84038938e-04] 1.9992160810616209
[ 8.99843192e+00  2.99843192e+00  6.99843192e+00  1.09984319e+01
 -1.56807786e-03  7.99843192e+00  9.99843192e+00 -1.56807786e-03
  3.99843192e+00  8.99843192e+00 -1.56807786e-03 -1.56807786e-03] 1.9984321621389225
[ 8.99764788e+00  2.99764788e+00  6.99764788e+00  1.09976479e+01
 -2.35211677e-03  7.99764788e+00  9.99764788e+00 -2.35211677e-03
  3.99764788e+00  8.99764788e+00 -2.35211677e-03 -2.35211677e-03] 1.997648243231905
[ 8.99686384e+00  2.99686384e+00  6.99686384e+00  1.09968638e+01
 -3.13615566e-03  7.99686384e+00  9.99686384e+00 -3.13615566e-03
  3.99686384e+00  8.99686384e+00 -3.13615566e-03 -3.13615566e-03] 1.9968643243405682
[ 8.99607981e+00  2.99607981e+00  6.99607981e+00  1.09960798e+01
 -3.92019454e-03  7.99607981e+00  9.99607981e+00 -3.92019454e-03
  3.9960798